# Preparing basic tools

In [3]:
import numpy as np
import cv2 as cv
# from sklearn import

If the image is 8-bit unsigned, it is displayed as is.
<br>If the image is 16-bit unsigned or 32-bit integer, the pixels are divided by 256. That is, the value range [0,255*256] is mapped to [0,255].
<br>If the image is 32-bit floating-point, the pixel values are multiplied by 255. **That is, the value range [0,1] is mapped to [0,255].**

In [259]:
def read_image(path : str):
    image = cv.imread(path)
    image //= 255 # see above
    grey_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return grey_img, np.shape(grey_img)

In [5]:
def identify_classes(X):
    return np.unique(X)

In [6]:
def display_image(title:str, image):
    cv.imshow(title, image)
    cv.waitKey()

In [196]:
def bruit_gauss(X, m, n, cl1, cl2, m1, sig1, m2, sig2):
    gaussian_class_1 = np.random.normal(m1, sig1, (m,n))
    gaussian_class_1[X == cl2] = 0
    gaussian_class_2 = np.random.normal(m2, sig2, (m,n)) # will probably have to use np.where class == {1,2}
    gaussian_class_2[X == cl1] = 0
    noisy_X = X + gaussian_class_1 + gaussian_class_2

    return noisy_X

# Puisqu'il s'agit de segmenter ... segmentons !

In [260]:
path = './images_BW/alfa2.bmp'
image, shape = read_image(path)
cl1, cl2 = identify_classes(image)
print(f'classe 1 : {cl1}, classe 2 : {cl2}')

classe 1 : 0, classe 2 : 1


In [273]:

noisy = bruit_gauss(image, shape[0], shape[1],cl1, cl2, 0, 0, 0, 0)

display_image('noisy', noisy)